# Лабораторная работа 6

Задачи:

* Провести классификацию русских текстов на несколько категорий. Лучше большой текст
* Провести предобработку текстов: нормализацию, лемматизацию и тд
* Сравнить эмбеддинги
* Попробовать несколько методов классификации (+косинусы)

Будем использовать Войну и мир, Краткую историю всего

In [1]:
# будем делить Войду и мир, Краткую историю всего на partsCount одинаковых частей
partsCount = 400
myTestSize = 0.2

textDir = "/home/alex/Downloads/bmstuML/texts/"

warFile = textDir + "voina_i_mir.txt"

istoriaFile = textDir + "kratkaya.txt"


with open(warFile, encoding="utf8") as f:
    war_text = f.read()
    
with open(istoriaFile, encoding="utf8") as f:
    istoria_text = f.read()

In [2]:
war_text[:300]

'\n\nЛев Толстой\n\nВойна и мир\n\nТома первый и второй\n\n\n\nВ.\xa0Шкловский «Война и мир» Льва Толстого\n\n\nЗамысел\n\nВ 1855 году появилось объявление об издании «Полярной звезды». На обложке книги в круге восходящего солнца были изображены пять портретов казненных декабристов; под портретами топор и подписано: «'

In [3]:
istoria_text[:300]

'\n«Краткая история почти всего на свете» Билла Брайсона — самая необычная энциклопедия из всех существующих! И это первая книга, которой была присуждена престижная европейская премия за вклад в развитие мировой науки имени Рене Декарта.\n\n\n\nСейчас, в начале 2003 года, я держу перед собой несколько стр'

### Токенизация

Выделим из текстов отдельные слова. Удалим ненужные "слова" (знаки препинания, лишние символы)


In [4]:
import nltk

nltk.download("punkt")
war_tokens = nltk.word_tokenize(war_text)

war_tokens[:15]

[nltk_data] Downloading package punkt to /home/alex/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


['Лев',
 'Толстой',
 'Война',
 'и',
 'мир',
 'Тома',
 'первый',
 'и',
 'второй',
 'В.',
 'Шкловский',
 '«',
 'Война',
 'и',
 'мир']

In [5]:
# второй текст
istoria_tokens = nltk.word_tokenize(istoria_text)

istoria_tokens[:15]

['«',
 'Краткая',
 'история',
 'почти',
 'всего',
 'на',
 'свете',
 '»',
 'Билла',
 'Брайсона',
 '—',
 'самая',
 'необычная',
 'энциклопедия',
 'из']

In [6]:
from snowballstemmer import RussianStemmer
stemmer = RussianStemmer()
stemmerResult = stemmer.stemWords(war_text.lower().split())

stemmerResult[:10]


['лев', 'толст', 'войн', 'и', 'мир', 'том', 'перв', 'и', 'втор', 'в.']

Как видим, слова получились "кривыми", простое обрезание окончания - не вариант. Воспользуемся лемматизатором.

In [7]:
import pymystem3

mystem = pymystem3.Mystem()

war_lemm = mystem.lemmatize(" ".join(war_tokens))

istoria_lemm = mystem.lemmatize(" ".join(istoria_tokens))



In [8]:
war_lemm[:10]

['лев', ' ', 'толстой', ' ', 'война', ' ', 'и', ' ', 'мир', ' ']

In [9]:
istoria_lemm[:10]

['« ', 'краткий', ' ', 'история', ' ', 'почти', ' ', 'все', ' ', 'на']

Посмотрим на морфологический анализ слов.

In [10]:
import pymorphy2
morph = pymorphy2.MorphAnalyzer()
morph.parse("свет")

[Parse(word='свет', tag=OpencorporaTag('NOUN,inan,masc,Sgtm sing,nomn'), normal_form='свет', score=0.51923, methods_stack=((DictionaryAnalyzer(), 'свет', 557, 0),)),
 Parse(word='свет', tag=OpencorporaTag('NOUN,inan,masc,Sgtm sing,accs'), normal_form='свет', score=0.423076, methods_stack=((DictionaryAnalyzer(), 'свет', 557, 4),)),
 Parse(word='свет', tag=OpencorporaTag('NOUN,anim,femn,Name plur,gent'), normal_form='света', score=0.01923, methods_stack=((DictionaryAnalyzer(), 'свет', 237, 8),)),
 Parse(word='свет', tag=OpencorporaTag('NOUN,anim,femn,Name plur,accs'), normal_form='света', score=0.01923, methods_stack=((DictionaryAnalyzer(), 'свет', 237, 10),)),
 Parse(word='свет', tag=OpencorporaTag('NOUN,anim,femn,Name sing,voct,Infr'), normal_form='света', score=0.01923, methods_stack=((DictionaryAnalyzer(), 'свет', 237, 13),))]

In [11]:
# Удалим знаки, пустые слова, числа

bad_chars = ["«", "»",".",",","—",":","1","2","3","4","5","6","7","8","9","0","?","-","[","]", "стр", "(", ")",
            ";", "X", "!", "издание", "…", "’","&"]

def containsStr(a,b):
    for i in b:
        if i in a:
            return True
    return False

def deleteWords(wordList):
    resultList = [item for item in wordList if len(item) > 1 ]
    resultList = [item for item in resultList if not containsStr(item, bad_chars) ]
    return resultList



war_lemm_clear = deleteWords(war_lemm)

istoria_lemm_clear = deleteWords(istoria_lemm)


In [12]:
war_lemm_clear[:10]

['лев',
 'толстой',
 'война',
 'мир',
 'том',
 'первый',
 'второй',
 'шкловский',
 'война',
 'мир']

In [13]:
istoria_lemm_clear[:10]

['краткий',
 'история',
 'почти',
 'все',
 'на',
 'свет',
 'билл',
 'брайсон',
 'самый',
 'необычный']

In [14]:
war_text_clear = " ".join(war_lemm_clear)
istoria_text_clear = " ".join(istoria_lemm_clear)

In [15]:
war_text_clear[:300]

'лев толстой война мир том первый второй шкловский война мир лев толстой замысел год появляться объявление об полярный звезда на обложка книга круг восходить солнце быть изображать пять портрет казнить декабрист под портрет топор подписывать июль год том помечать день казнь декабрист над заглавие туч'

In [16]:
# сохраним текста

# with open(textDir+ '1.txt', 'w') as file:
#     file.write(war_text_clear)
    
# with open(textDir+ '2.txt', 'w') as file:
#     file.write(istoria_text_clear)  


In [17]:
# функция разделения на части
def chunkify(lst,n):
    return [lst[i::n] for i in range(n)]

In [18]:
from string import punctuation

# def lemmatize(input_text):
#     tokens = nltk.word_tokenize(input_text)
#     normed_tokens = [morph.parse(s)[0].normal_form for s in tokens]
#     # исключии стоп слова
#     normed_tokens = deleteWords(normed_tokens)
#     return ' '.join(normed_tokens)

In [19]:
import os
import pandas as pd
import numpy as np


# # nogroToByM uycTow naTtadpevim
# df = pd.DataFrame(columns=['text', 'class'])

# dir0 = textDir + "/0"
# dir1 = textDir + "/1"

# for filename in os.listdir(dir0) :
#     with open(os.path.join(dir0, filename), encoding='utf8') as file:
#         contents = lemmatize(file.read())

#     df = df.append(pd.Series({'text': contents, 'class': 0}), ignore_index=True)



In [20]:

# for filename in os.listdir(dir1) :
#     with open(os.path.join(dir1, filename), encoding='utf8') as file:
#         contents = lemmatize(file.read())

#     df = df.append(pd.Series({'text': contents, 'class': 1}), ignore_index=True)

In [21]:

war_parts = chunkify(war_lemm_clear, partsCount)
istoria_parts = chunkify(istoria_lemm_clear, partsCount)

istoria_parts[1][:10]

['история',
 'флэннерить',
 'точно',
 'яйцо',
 'же',
 'помогать',
 'содержать',
 'как',
 'шестидесятый',
 'ускоритель']

In [22]:
# объединяем слова в частях
# на выходе - список из текстов
def joinWordsToTextList(wordsLists):
    result = []
    for i in wordsLists:
        oneText = " ".join(i)
        result.append(oneText)
    return result


war_texts = joinWordsToTextList(war_parts)
istoria_texts = joinWordsToTextList(istoria_parts)

In [23]:
# war_texts[1][:300]

In [24]:
df = pd.DataFrame(columns=["text","class"])


for i in war_texts:
    df = df.append(pd.Series({"text":i, "class":0}), ignore_index=True)

# у второго типа текстов класс == 1
for i in istoria_texts:
    df = df.append(pd.Series({"text":i, "class":1}), ignore_index=True)
    

print("war_texts size:"+str(len(war_texts[3])), "\tistoria_text size:"+str(len(istoria_texts[3])))


war_texts size:3496 	istoria_text size:2585


In [25]:
df

,text,class
0,лев приобретать понимать состоять под как нет ...,0
1,толстой он сила только влияние бы мать борьба ...,0
2,война великий народ то исторический смерть тол...,0
3,мир слава слабость что событие она конец помощ...,0
4,том который декабрист мы война новый давать не...,0
...,...,...
795,рич куча отращивать добела предполагать малень...,1
796,из создавать плавник шар что что склоняться от...,1
797,музей живой конечность из мочь крошечный велич...,1
798,штат существо щеголять железо сам точка млрд б...,1


In [26]:
# разбив на трейн и тест
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(df["text"], df["class"], test_size=myTestSize, random_state=42)

In [27]:
X_train

264    хотеть год роман отражение андрей тот один сил...
615    естественный небольшой век этот все размышлени...
329    стиль быть история осуществляться взятка котор...
342    восторг причем думать выражать как становиться...
394    севастополь пьер не не пьер хотеть портфель ко...
                             ...                        
71     свой николай эпоха все герой любовь роман умол...
106    весь наташа уже описывать выглядеть роман суд ...
270    так больший что положение гореть чтобы последо...
435    перед зеландия другой эволюция что неприятно и...
102    полярный это этот мир четкость конфликт жизнь ...
Name: text, Length: 640, dtype: object

In [28]:
X_test

696    национальный быть что просто самый знание прав...
667    штат что курьезность послужить область уйма об...
63     как для наполеоновский бы во роман оливер скор...
533    этот мочь наполняться вознаграждать содержать ...
66     свой пьер нестарый отсутствие счастие реалисти...
                             ...                        
589    лондон действие тот сочетание часть находиться...
798    штат существо щеголять железо сам точка млрд б...
744    даллас или из собственный что направление мате...
513    здесь поздравлять атом все здоровый мой столь ...
670    оффенбур приниматься что он оставаться вопрос ...
Name: text, Length: 160, dtype: object

## Bag-of-Words-эмбеддинг
на всех данных обучние - чтоб все слова были учтены?

In [29]:
from sklearn.feature_extraction.text import CountVectorizer

bof_vect = CountVectorizer()
bof_vect.fit(np.hstack([X_train, X_test]))
bof_train = bof_vect.transform(X_train)
bof_test = bof_vect.transform(X_test)

In [30]:
# пространство признаков

bof_train.toarray()

array([[0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       ...,
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 1, 0, 0],
       [0, 0, 0, ..., 0, 0, 0]])

In [31]:
bof_train.toarray().shape

(640, 23015)

## TF-IDF-Эмбеддинг

In [32]:
from sklearn.feature_extraction.text import TfidfVectorizer

tfidf_vect = TfidfVectorizer()
tfidf_vect.fit(np.hstack([X_train, X_test]))
tfidf_train = tfidf_vect.transform(X_train)
tfidf_test = tfidf_vect.transform(X_test)

In [57]:
tfidf_train.toarray()

array([[0.        , 0.        , 0.        , ..., 0.        , 0.        ,
        0.        ],
       [0.        , 0.        , 0.        , ..., 0.        , 0.        ,
        0.        ],
       [0.        , 0.        , 0.        , ..., 0.        , 0.        ,
        0.        ],
       ...,
       [0.        , 0.        , 0.        , ..., 0.        , 0.        ,
        0.        ],
       [0.        , 0.        , 0.        , ..., 0.05868196, 0.        ,
        0.        ],
       [0.        , 0.        , 0.        , ..., 0.        , 0.        ,
        0.        ]])

In [34]:
tfidf_train.toarray().shape

(640, 23015)

## Word2vec-эмбеддинг
Немного другой формат данных

In [35]:
from gensim.models import Word2Vec

X_train_w2v = X_train.apply(str.split)
X_test_w2v = X_test.apply(str.split)
w2v_vect = Word2Vec(np.hstack([X_train_w2v, X_test_w2v]), size=160, min_count=10)

In [36]:
X_train_w2v

264    [хотеть, год, роман, отражение, андрей, тот, о...
615    [естественный, небольшой, век, этот, все, разм...
329    [стиль, быть, история, осуществляться, взятка,...
342    [восторг, причем, думать, выражать, как, стано...
394    [севастополь, пьер, не, не, пьер, хотеть, порт...
                             ...                        
71     [свой, николай, эпоха, все, герой, любовь, ром...
106    [весь, наташа, уже, описывать, выглядеть, рома...
270    [так, больший, что, положение, гореть, чтобы, ...
435    [перед, зеландия, другой, эволюция, что, непри...
102    [полярный, это, этот, мир, четкость, конфликт,...
Name: text, Length: 640, dtype: object

In [58]:
w2v_vect.most_similar(positive="год")

/home/alex/.local/lib/python3.6/site-packages/ipykernel_launcher.py:1: DeprecationWarning: Call to deprecated `most_similar` (Method will be removed in 4.0.0, use self.wv.most_similar() instead).
  """Entry point for launching an IPython kernel.


[('земля', 0.9965345859527588),
 ('если', 0.9844806790351868),
 ('вид', 0.9637594223022461),
 ('мы', 0.9588470458984375),
 ('более', 0.9585084915161133),
 ('почти', 0.9536319971084595),
 ('самый', 0.9487262964248657),
 ('существо', 0.9473338723182678),
 ('даже', 0.9460662603378296),
 ('теория', 0.9353926181793213)]

In [38]:
w2v_vect.most_similar(negative=["существо"])

/home/alex/.local/lib/python3.6/site-packages/ipykernel_launcher.py:1: DeprecationWarning: Call to deprecated `most_similar` (Method will be removed in 4.0.0, use self.wv.most_similar() instead).
  """Entry point for launching an IPython kernel.


[('князь', 0.1362890601158142),
 ('ты', 0.13006210327148438),
 ('наташа', 0.12526768445968628),
 ('пьер', 0.12225104868412018),
 ('сказать', 0.11963337659835815),
 ('андрей', 0.07721569389104843),
 ('лицо', 0.06798352301120758),
 ('ростов', 0.06383147090673447),
 ('она', 0.038992077112197876),
 ('да', 0.033359773457050323)]

In [39]:
# как представлено слово земля
w2v_vect["земля"]

/home/alex/.local/lib/python3.6/site-packages/ipykernel_launcher.py:2: DeprecationWarning: Call to deprecated `__getitem__` (Method will be removed in 4.0.0, use self.wv.__getitem__() instead).
  


array([-2.94823050e-01,  2.34902352e-01,  1.55353872e-02,  5.65707088e-01,
       -2.38489166e-01, -2.75986701e-01, -2.88902745e-02, -7.14717060e-02,
        6.77703381e-01,  2.78774463e-02,  2.15267301e-01,  3.32254797e-01,
        2.11352706e-01, -4.40875217e-02,  3.07136983e-01,  3.06816585e-02,
        1.28426969e-01, -2.83240199e-01,  4.32906955e-01, -8.17743242e-01,
       -6.81228042e-01,  4.30307627e-01,  2.20073387e-01, -5.52327394e-01,
       -2.69692183e-01,  2.37412825e-01, -1.34634614e-01, -4.51808989e-01,
        1.76651314e-01,  2.54281998e-01,  1.71549141e-01, -6.28879905e-01,
       -4.23199117e-01,  2.15165064e-01,  7.59299323e-02, -2.52604727e-02,
       -3.77696119e-02, -7.06914425e-01,  7.25381315e-01, -5.20020239e-02,
        1.88641295e-01, -2.94054478e-01, -1.09110475e-02,  1.45482093e-01,
       -1.06977308e+00,  3.84849906e-02, -5.35340756e-02, -1.08878374e+00,
        9.58952159e-02,  3.37399483e-01, -1.87360674e-01, -4.88749772e-01,
        1.68457121e-01, -

In [59]:
# усредним векторы слов
def text2vec(text):
    vecs = []
    for word in text:
        try:
            vecs.append(w2v_vect[word])
        except KeyError:
            pass
    return np.sum(vecs, axis=0) / len(vecs)

    
    
w2v_train = X_train_w2v.apply(text2vec)
w2v_test = X_test_w2v.apply(text2vec)

w2v_train

/home/alex/.local/lib/python3.6/site-packages/ipykernel_launcher.py:6: DeprecationWarning: Call to deprecated `__getitem__` (Method will be removed in 4.0.0, use self.wv.__getitem__() instead).
  


264    [0.40148202, -0.13114636, 0.08417224, 0.093339...
615    [0.1996644, -0.031440105, 0.062310632, 0.19608...
329    [0.40392992, -0.13218908, 0.084841885, 0.09331...
342    [0.43053618, -0.14531736, 0.08748263, 0.079589...
394    [0.42451337, -0.14174514, 0.086844474, 0.08285...
                             ...                        
71     [0.4144408, -0.13631926, 0.08605696, 0.0918676...
106    [0.40112707, -0.13162608, 0.083633736, 0.09032...
270    [0.41912636, -0.13982041, 0.08575793, 0.082934...
435    [0.20149308, -0.033317048, 0.061888132, 0.1915...
102    [0.39735636, -0.1291441, 0.08374218, 0.0954770...
Name: text, Length: 640, dtype: object

In [60]:
w2v_train = np.dstack(w2v_train)[0]

In [61]:
w2v_train

array([[ 0.40148202,  0.1996644 ,  0.40392992, ...,  0.41912636,
         0.20149308,  0.39735636],
       [-0.13114636, -0.03144011, -0.13218908, ..., -0.13982041,
        -0.03331705, -0.1291441 ],
       [ 0.08417224,  0.06231063,  0.08484188, ...,  0.08575793,
         0.06188813,  0.08374218],
       ...,
       [ 0.0959761 ,  0.08545016,  0.09635032, ...,  0.09662557,
         0.08453091,  0.09551174],
       [ 0.11246458,  0.03771376,  0.11333334, ...,  0.11910814,
         0.0388655 ,  0.11088072],
       [-0.28216082, -0.21835066, -0.28337517, ..., -0.28675652,
        -0.21779577, -0.2811363 ]], dtype=float32)

In [43]:
w2v_train.shape

(160, 640)

## Классификация текстов
Для каждого эмбеддинга вычислим по два суммарных вектора - для текстов истории и мира(вона и мир)

#### Для Bag-of-Words

In [62]:
# средний текст войны и мира
mir_mean_bof = np.sum( bof_train[y_train == 0], axis=0 )
mir_mean_bof.shape

(1, 23015)

In [45]:
# средний текст истории
istoria_mean_bof = np.sum( bof_train[y_train == 1], axis=0 )
istoria_mean_bof.shape

(1, 23015)

#### Для TF-IDF

In [46]:
mir_mean_tfidf = np.sum(tfidf_train[y_train == 0], axis=0 )

istoria_mean_tfidf = np.sum(tfidf_train[y_train == 1], axis=0 )


mir_mean_tfidf.shape

(1, 23015)

#### Для Word2Vec

In [47]:

mir_mean_w2v = np.sum(w2v_train[:,y_train==0], axis=1)

istoria_mean_w2v = np.sum(w2v_train[:,y_train==1], axis=1)

mir_mean_w2v.shape


(160,)

In [48]:
mir_mean_w2v

array([ 130.77219  ,  -43.02624  ,   27.191893 ,   28.78016  ,
        105.58279  ,  -34.633293 ,  -38.023174 ,    7.979184 ,
        -62.360767 ,  -64.91503  ,   12.221597 ,  -36.243176 ,
        -32.004898 ,   13.283019 ,  146.64664  ,   24.513514 ,
        -10.21763  ,    9.083516 ,   10.542308 ,   10.454503 ,
        -15.301132 ,  -51.942078 ,   58.39855  ,  -47.762318 ,
       -152.27715  ,   32.266457 ,   43.22663  ,  -48.80561  ,
         77.453735 ,   84.463875 ,   -5.9783936,   74.45671  ,
        -24.969944 ,  196.47798  ,   70.5737   ,   47.171608 ,
        -30.887072 ,   53.099068 ,   30.80424  ,   89.85188  ,
         31.40989  ,   43.275963 ,   31.276043 ,  -30.753492 ,
          5.3987265,  160.50761  , -117.725784 ,  -66.904564 ,
        -33.9499   ,  -24.136772 ,  -38.711174 ,   69.45814  ,
         19.408567 ,  -36.036316 ,  -10.918736 ,  -87.85166  ,
        -77.32976  ,  -47.549763 ,   14.765736 , -121.93524  ,
         93.55587  ,  -51.77681  , -104.660576 ,  -24.3

Построим датафреймы с результатами классификаций тестовых текстов. Решение - по косинусному расстоянию.

In [63]:
from scipy.spatial.distance import cosine

bof_mir = np.apply_along_axis(cosine, 1, bof_test.toarray(), v=mir_mean_bof)
bof_istoria = np.apply_along_axis(cosine, 1, bof_test.toarray(), v=istoria_mean_bof)

bof_results = pd.DataFrame ([
    bof_mir,
    bof_istoria,
    np.maximum(bof_mir, bof_istoria) == bof_istoria,
    y_test
], index=["mir", "istoria", "predict", "class"]).T.astype(np.float)

bof_results

,mir,istoria,predict,class
0,0.482883,0.311288,0.0,1.0
1,0.429743,0.289305,0.0,1.0
2,0.198570,0.286129,1.0,0.0
3,0.426814,0.311105,0.0,1.0
4,0.153879,0.312206,1.0,0.0
...,...,...,...,...
155,0.486079,0.293769,0.0,1.0
156,0.330891,0.272752,0.0,1.0
157,0.447075,0.276866,0.0,1.0
158,0.447795,0.305260,0.0,1.0


In [64]:
from sklearn.metrics import accuracy_score

accuracy_score(bof_results["predict"], bof_results["class"])

0.0

In [65]:
tfidf_mir = np.apply_along_axis(cosine, 1, tfidf_test.toarray(), v=mir_mean_tfidf)
tfidf_istoria = np.apply_along_axis(cosine, 1, tfidf_test.toarray(), v=istoria_mean_tfidf)

tfidf_results = pd.DataFrame ([
    tfidf_mir,
    tfidf_istoria,
    np.maximum(tfidf_mir, tfidf_istoria) == tfidf_istoria,
    y_test
], index=["mir", "istoria", "predict", "class"]).T.astype (np.float)

tfidf_results

,mir,istoria,predict,class
0,0.801804,0.649217,0.0,1.0
1,0.772935,0.650867,0.0,1.0
2,0.503860,0.670587,1.0,0.0
3,0.781306,0.656577,0.0,1.0
4,0.488976,0.678448,1.0,0.0
...,...,...,...,...
155,0.790895,0.634754,0.0,1.0
156,0.726309,0.614806,0.0,1.0
157,0.782911,0.639474,0.0,1.0
158,0.784943,0.664064,0.0,1.0


In [52]:
accuracy_score(tfidf_results["predict"], tfidf_results["class"])

0.0

In [53]:
# w2v

w2v_mir = np.apply_along_axis(cosine, 0, w2v_test, v=mir_mean_w2v)
w2v_istoria = np.apply_along_axis(cosine, 0, w2v_test, v=istoria_mean_w2v)

w2v_results = pd.DataFrame ([
    tfidf_mir,
    tfidf_istoria,
    np.maximum(w2v_mir, w2v_istoria) == w2v_istoria,
    y_test
], index=["mir", "istoria", "predict", "class"]).T.astype (np.float)

w2v_results



,mir,istoria,predict,class
0,0.801804,0.649217,1.0,1.0
1,0.772935,0.650867,0.0,1.0
2,0.503860,0.670587,1.0,0.0
3,0.781306,0.656577,1.0,1.0
4,0.488976,0.678448,1.0,0.0
...,...,...,...,...
155,0.790895,0.634754,1.0,1.0
156,0.726309,0.614806,0.0,1.0
157,0.782911,0.639474,1.0,1.0
158,0.784943,0.664064,1.0,1.0


In [54]:
accuracy_score(w2v_results["predict"], w2v_results["class"])

0.4625

#### Попробуем применить классическую модель поверх эмбеддингов

In [55]:
from sklearn.ensemble import RandomForestClassifier

RandomForestClassifier().fit(tfidf_train.toarray(), y_train.tolist()).score(tfidf_test.toarray(), y_test.tolist())

1.0

In [56]:
RandomForestClassifier().fit(bof_train.toarray(), y_train.tolist()).score(bof_test.toarray(), y_test.tolist())

1.0